* http://potatoggg.tistory.com/92
* https://github.com/mutux/suffix-tries/blob/master/suffixtrie.py
* http://www.cs.jhu.edu/~langmea/resources/lecture_notes/tries_and_suffix_tries.pdf
* https://github.com/vancanhuit/simple-data-compression
* https://github.com/etcwilde/lz_compression
* http://faculty.kfupm.edu.sa/ICS/jauhar/ics202/Unit31_LZ78.ppt (얘가 제일 잘 정리)
* https://www.cs.helsinki.fi/u/puglisi/dct2015/slides7.pdf (Trie가 있어...)

## 1. Basic LZ78 (1555051, 1271185, 81.75%)

In [1]:
class LZ78_dict:
    
    def readfile(self, filename):
        try:
            f = open(filename, 'r')
            data = f.readlines()
            f.close()
        except UnicodeDecodeError:
            # 입력 스트림과 출력 스트림을 연다
            input = open(filename, "rt", encoding="utf-16")
            data = ''

            # 유니코드 데이터 조각들을 스트리밍한다
            with input:
                while True:
                    # 데이터 조각을 읽고
                    chunk = input.read(4096)
                    if not chunk:
                        break
                    # 수직 탭을 삭제한다
                    chunk = chunk.replace("\u000B", "")
                    # 데이터 조각을 쓴다
                    data += chunk

        return data
    
    def compress(self, origin_file, compressed_file):
        import struct 
        data = ''.join(self.readfile(origin_file))
        encoded_data = self.encoding(data)
        
        binfile = open(compressed_file, 'wb') 
        for idx, ch in encoded_data:
                
            data = struct.pack('Ic', idx, ch.encode()) 
            binfile.write(data) 
        binfile.close()
        
    def decompress(self, compressed_file, decompressed_file):
        import struct 
        binfile = open(compressed_file, 'rb')
        intsize = struct.calcsize('Ic')
        encoded_data = []

        while True:
            binary = binfile.read(intsize)
            if binary == b'': 
                break 
            encoded_data.append(struct.unpack('Ic', binary))
            
        encoded_data = [(d[0], d[1].decode()) for d in encoded_data]
        
        decoded_data = self.decoding(encoded_data)
        
        f = open(decompressed_file, 'w')
        f.write(decoded_data)
        f.close()
        
    def encoding(self, data):
        import collections
        encode_dict = collections.OrderedDict()
        out = []
        out2 = []
        key = ''
        
        for i, c in enumerate(data):
            key += c
            if key not in encode_dict:
                out.append((encode_dict[key[:-1]] if len(key) > 1 else 0, c))
                encode_dict[key] = len(encode_dict)+1
                key = ''
                
        if key != '': out.append((encode_dict[key], ''))

        return out
    
    def decoding(self, data):
        d = []
        p = ''

        for (w, c) in data: d.append(c if w == 0 else d[w-1] + c)

        return ''.join(d)

* Check encoding/decoding logic

In [2]:
lz78 = LZ78_dict()

In [3]:
text = 'i love computer science'

In [4]:
encoded_text = lz78.encoding(text)
decoded_text = lz78.decoding(encoded_text)
decoded_text

'i love computer science'

In [5]:
data = ['ABBCBCABABCAABCAAB', 'BABAABRRRA', 'AAAAAAAAA']

for origin_text in data:
    lz78_ = LZ78_dict()
    encoded_text = lz78_.encoding(origin_text)
    decoded_text = lz78_.decoding(encoded_text)
    print(origin_text, encoded_text, decoded_text, origin_text == decoded_text)
    print('--------')

ABBCBCABABCAABCAAB [(0, 'A'), (0, 'B'), (2, 'C'), (3, 'A'), (2, 'A'), (4, 'A'), (6, 'B')] ABBCBCABABCAABCAAB True
--------
BABAABRRRA [(0, 'B'), (0, 'A'), (1, 'A'), (2, 'B'), (0, 'R'), (5, 'R'), (2, '')] BABAABRRRA True
--------
AAAAAAAAA [(0, 'A'), (1, 'A'), (2, 'A'), (3, '')] AAAAAAAAA True
--------


* Test data compression and Calculate compression ratio

In [6]:
lz78 = LZ78_dict()

In [7]:
lz78.compress('infile.txt', 'compress.lz')

In [8]:
lz78.decompress('compress.lz', 'restore.txt')

In [9]:
import os

In [10]:
print('File name: infile.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%' % 
('{:,}'.format(os.path.getsize('./infile.txt')), '{:,}'.format(os.path.getsize('./compress.lz')), 
 os.path.getsize('./compress.lz')/os.path.getsize('./infile.txt')*100))

File name: infile.txt, 
Origin file size: 1,555,051Byte, 
Compressed size: 1,271,185Byte, 
Compression ratio: 81.745550%


----

## 2. 4byte LZ78 (1555051, 1016948, 65.4%)

In [11]:
class LZ78_4byte:
    
    def readfile(self, filename):
        try:
            f = open(filename, 'r')
            data = f.readlines()
            f.close()
        except UnicodeDecodeError:
            # 입력 스트림과 출력 스트림을 연다
            input = open(filename, "rt", encoding="utf-16")
            data = ''

            # 유니코드 데이터 조각들을 스트리밍한다
            with input:
                while True:
                    # 데이터 조각을 읽고
                    chunk = input.read(4096)
                    if not chunk:
                        break
                    # 수직 탭을 삭제한다
                    chunk = chunk.replace("\u000B", "")
                    # 데이터 조각을 쓴다
                    data += chunk

        return data
    
    def compress(self, origin_file, compressed_file):
        import struct 
        
        data = ''.join(self.readfile(origin_file))
        encoded_data = self.encoding(data)
        
        maxidx = max([t[0] for t in encoded_data])
        
        if maxidx < 255:
            bintype = 'Bc'
        elif maxidx < 65535:
            bintype = 'Hc'
        elif maxidx < 16777215:
            bintype = '4byte'
        else:
            bintype = 'Ic'
        
        binfile = open(compressed_file, 'wb')
        binfile.write(struct.pack('c', bintype[:1].encode()))
        for idx, ch in encoded_data:
            if len(ch) == 0:
                break
            if bintype == '4byte':
                data = struct.pack('Ic', idx, ch.encode())
                binfile.write(b''.join([data[0:3], data[4:]])) 
            else:
                data = struct.pack(bintype, idx, ch.encode())
                binfile.write(data) 
        binfile.close()
        
    def decompress(self, compressed_file, decompressed_file):
        import struct 
        binfile = open(compressed_file, 'rb')
        a = binfile.read(1)
        bintype = struct.unpack('c', a)[0].decode()
        encoded_data = []

        while True:
            if bintype == '4':
                binary = binfile.read(struct.calcsize('Ic') - 1)
                if binary == b'': break 
                encoded_data.append(struct.unpack('Ic', b''.join([binary[0:3], b'\x00', binary[3:]])))
            else:
                binary = binfile.read(struct.calcsize('%sc' % bintype))
                if binary == b'': break 
                encoded_data.append(struct.unpack('%sc' % bintype, binary))
            
        encoded_data = [(d[0], d[1].decode()) for d in encoded_data]
        
        decoded_data = self.decoding(encoded_data)
        
        f = open(decompressed_file, 'w')
        f.write(decoded_data)
        f.close()
        
    def encoding(self, data):
        import collections
        encode_dict = collections.OrderedDict()
        out = []
        out2 = []
        key = ''
        
        for i, c in enumerate(data):
            key += c
            if key not in encode_dict:
                out.append((encode_dict[key[:-1]] if len(key) > 1 else 0, c))
                encode_dict[key] = len(encode_dict)+1
                key = ''
                
        if key != '': out.append((encode_dict[key], ''))

        return out
    
    def decoding(self, data):
        d = []
        p = ''

        for (w, c) in data: d.append(c if w == 0 else d[w-1] + c)

        return ''.join(d)

* Test data compression and Calculate compression ratio

In [12]:
lz78 = LZ78_4byte()

In [13]:
lz78.compress('infile.txt', 'compress.lz')

In [14]:
lz78.decompress('compress.lz', 'restore.txt')

In [15]:
import os

In [16]:
print('File name: infile.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%' % 
('{:,}'.format(os.path.getsize('./infile.txt')), '{:,}'.format(os.path.getsize('./compress.lz')), 
 os.path.getsize('./compress.lz')/os.path.getsize('./infile.txt')*100))

File name: infile.txt, 
Origin file size: 1,555,051Byte, 
Compressed size: 1,016,949Byte, 
Compression ratio: 65.396505%


* Test each type of byte
    * infile2.txt: 2byte
    * infile3.txt: 3byte
    * infile.txt: 4byte

In [17]:
for i in range(2, 4):
    lz78 = LZ78_4byte()
    lz78.compress('infile%d.txt' % i, 'compress%d.lz' % i)
    lz78.decompress('compress%d.lz' % i, 'restore%d.txt' % i)
    print('Test %dbyte' % i)
    print('File name: infile%d.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%\n' % 
          (i, '{:,}'.format(os.path.getsize('./infile%d.txt' % i)), '{:,}'.format(os.path.getsize('./compress%d.lz' % i)), 
           os.path.getsize('./compress%d.lz' % i)/os.path.getsize('./infile%d.txt' % i)*100))

Test 2byte
File name: infile2.txt, 
Origin file size: 518Byte, 
Compressed size: 445Byte, 
Compression ratio: 85.907336%

Test 3byte
File name: infile3.txt, 
Origin file size: 2,452Byte, 
Compressed size: 2,434Byte, 
Compression ratio: 99.265905%



----

## 3. Switch LZ78 (1555051, 951156, 61.17%)

In [64]:
class LZ78_switch:
    
    def readfile(self, filename):
        try:
            f = open(filename, 'r')
            data = f.readlines()
            f.close()
        except UnicodeDecodeError:
            # 입력 스트림과 출력 스트림을 연다
            input = open(filename, "rt", encoding="utf-16")
            data = ''

            # 유니코드 데이터 조각들을 스트리밍한다
            with input:
                while True:
                    # 데이터 조각을 읽고
                    chunk = input.read(4096)
                    if not chunk:
                        break
                    # 수직 탭을 삭제한다
                    chunk = chunk.replace("\u000B", "")
                    # 데이터 조각을 쓴다
                    data += chunk

        return data
    
    def compress(self, origin_file, compressed_file):
        import struct 
        
        data = ''.join(self.readfile(origin_file))
        encoded_data = self.encoding(data)
                
        binfile = open(compressed_file, 'wb')
        for i, (idx, ch) in enumerate(encoded_data):
            if len(ch) == 0:
                break
                                        
            if i <= 255:
                binsize = 2
                data = struct.pack('Bc', idx, ch.encode())
                binfile.write(data) 
            elif i <= 65535:
                binsize = 3
                data = struct.pack('Hc', idx, ch.encode())
                binfile.write(data) 
            elif i <= 16777215:
                binsize = 4
                data = struct.pack('Ic', idx, ch.encode())
                binfile.write(b''.join([data[0:3], data[4:]])) 
            else:
                binsize = 5
                data = struct.pack('Ic', idx, ch.encode())
                binfile.write(data)            
            
        binfile.close()
        
    def decompress(self, compressed_file, decompressed_file):
        import struct 
        binfile = open(compressed_file, 'rb')
        bintype = {2: 'Bc', 3: 'Hc', 4: 'Ic', 5: 'Ic'}
               
        encoded_data = []
        seq = 0
        
        while True:
            if seq <= 255:
                binsize = 2
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], binary))
            elif seq <= 65535:
                binsize = 3
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], binary))
            elif seq <= 16777215:
                binsize = 4
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], b''.join([binary[0:3], b'\x00', binary[3:]])))
            else:
                binsize = 5
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], binary))
                
            seq += 1
        
        encoded_data = [(d[0], d[1].decode()) for d in encoded_data]
        decoded_data = self.decoding(encoded_data)
        
        f = open(decompressed_file, 'w')
        f.write(decoded_data)
        f.close()
        
    def encoding(self, data):
        import collections
        encode_dict = collections.OrderedDict()
        out = []
        out2 = []
        key = ''
        
        for i, c in enumerate(data):
            key += c
            if key not in encode_dict:
                out.append((encode_dict[key[:-1]] if len(key) > 1 else 0, c))
                encode_dict[key] = len(encode_dict)+1
                key = ''
                
        if key != '': out.append((encode_dict[key], ''))

        return out
    
    def decoding(self, data):
        d = []
        p = ''

        for (w, c) in data: d.append(c if w == 0 else d[w-1] + c)

        return ''.join(d)

* Test data compression and Calculate compression ratio

In [19]:
lz78 = LZ78_switch()

In [20]:
lz78.compress('infile.txt', 'compress.lz')

In [21]:
lz78.decompress('compress.lz', 'restore.txt')

In [22]:
import os

In [23]:
print('File name: infile.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%' % 
('{:,}'.format(os.path.getsize('./infile.txt')), '{:,}'.format(os.path.getsize('./compress.lz')), 
 os.path.getsize('./compress.lz')/os.path.getsize('./infile.txt')*100))

File name: infile.txt, 
Origin file size: 1,555,051Byte, 
Compressed size: 951,156Byte, 
Compression ratio: 61.165582%


* Test each type of byte
    * infile2.txt: 2byte
    * infile3.txt: 3byte
    * infile.txt: 4byte

In [24]:
for i in range(2, 4):
    lz78 = LZ78_switch()
    lz78.compress('infile%d.txt' % i, 'compress%d.lz' % i)
    lz78.decompress('compress%d.lz' % i, 'restore%d.txt' % i)
    print('Test %dbyte' % i)
    print('File name: infile%d.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%\n' % 
          (i, '{:,}'.format(os.path.getsize('./infile%d.txt' % i)), '{:,}'.format(os.path.getsize('./compress%d.lz' % i)), 
           os.path.getsize('./compress%d.lz' % i)/os.path.getsize('./infile%d.txt' % i)*100))

Test 2byte
File name: infile2.txt, 
Origin file size: 518Byte, 
Compressed size: 444Byte, 
Compression ratio: 85.714286%

Test 3byte
File name: infile3.txt, 
Origin file size: 2,452Byte, 
Compressed size: 2,177Byte, 
Compression ratio: 88.784666%



* Unicode File Test

In [25]:
lz78 = LZ78_switch()
lz78.compress('infile_unicode.txt', 'compress_unicode.lz')
lz78.decompress('compress_unicode.lz', 'restore_unicode.txt')

print('File name: infile_unicode.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%' % 
('{:,}'.format(os.path.getsize('./infile_unicode.txt')), '{:,}'.format(os.path.getsize('./compress_unicode.lz')), 
 os.path.getsize('./compress_unicode.lz')/os.path.getsize('./infile_unicode.txt')*100))

File name: infile_unicode.txt, 
Origin file size: 1,038Byte, 
Compressed size: 444Byte, 
Compression ratio: 42.774566%


----

## 4. Switch LZ78 with Trie (1555051, 951156, 61.17%)

In [119]:
class LZ78_trie:
    
    def readfile(self, filename):
        try:
            f = open(filename, 'r')
            data = f.readlines()
            f.close()
        except UnicodeDecodeError:
            # 입력 스트림과 출력 스트림을 연다
            input = open(filename, "rt", encoding="utf-16")
            data = ''

            # 유니코드 데이터 조각들을 스트리밍한다
            with input:
                while True:
                    # 데이터 조각을 읽고
                    chunk = input.read(4096)
                    if not chunk:
                        break
                    # 수직 탭을 삭제한다
                    chunk = chunk.replace("\u000B", "")
                    # 데이터 조각을 쓴다
                    data += chunk

        return data
    
    def compress(self, origin_file, compressed_file):
        import struct 
        
        data = ''.join(self.readfile(origin_file))
        encoded_data = self.encoding(data)
                
        binfile = open(compressed_file, 'wb')
        for i, (idx, ch) in enumerate(encoded_data):
                        
            if i <= 255:
#                 binsize = 2
                data = struct.pack('Bc', idx, ch.encode() if len(ch) > 0 else b'\x00')
                binfile.write(data) 
            elif i <= 65535:
#                 binsize = 3
                data = struct.pack('Hc', idx, ch.encode() if len(ch) > 0 else b'\x00')
                binfile.write(data) 
            elif i <= 16777215:
#                 binsize = 4
                data = struct.pack('Ic', idx, ch.encode() if len(ch) > 0 else b'\x00')
                binfile.write(b''.join([data[0:3], data[4:]])) 
            else:
#                 binsize = 5
                data = struct.pack('Ic', idx, ch.encode() if len(ch) > 0 else b'\x00')
                binfile.write(data)            
            
        binfile.close()
        
    def decompress(self, compressed_file, decompressed_file):
        import struct 
        binfile = open(compressed_file, 'rb')
        bintype = {2: 'Bc', 3: 'Hc', 4: 'Ic', 5: 'Ic'}
               
        encoded_data = []
        seq = 0
        
        while True:
            if seq <= 255:
                binsize = 2
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], binary))
            elif seq <= 65535:
                binsize = 3
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], binary))
            elif seq <= 16777215:
                binsize = 4
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], b''.join([binary[0:3], b'\x00', binary[3:]])))
            else:
                binsize = 5
                binary = binfile.read(binsize)
                if binary == b'': break 
                encoded_data.append(struct.unpack(bintype[binsize], binary))
                
            seq += 1
        
        if encoded_data[-1][1] == b'\x00':
            encoded_data[-1] = (encoded_data[-1][0], b'')
            
        encoded_data = [(d[0], d[1].decode()) for d in encoded_data]
        decoded_data = self.decoding(encoded_data)
        
        f = open(decompressed_file, 'w')
        f.write(decoded_data)
        f.close()
        
    def getEmptyTrie(self, data):
        available_characters = [i for i in range(ord('a'), ord('z')+1)] + \
            [i for i in range(ord('A'), ord('Z')+1)] + \
            [i for i in range(ord('0'), ord('9')+1)] + \
            [ord('!'), ord('?'), ord(' '), ord(','), ord('.'), ord(':'), ord(';'), ord('\n')]
        available_characters_dict = {}
        for i, c in enumerate(available_characters): available_characters_dict[chr(c)] = i
            
        return [[-1]*70 for _ in range(len(data))], available_characters_dict
        
    def encoding(self, data):
        out = []
        trie, available_characters_dict = self.getEmptyTrie(data)
        presentState = 0
        state = 1
        
        key = ''
        for c in data:
            ch = available_characters_dict[c]
            if trie[presentState][ch] == -1:
                trie[presentState][ch] = state
                out.append((presentState, c))
                presentState = 0
                state += 1
            else:
                presentState = trie[presentState][ch]
        
        if presentState > 0: out.append((presentState, ''))

        return out
    
    def decoding(self, data):
        d = []

        for (w, c) in data: d.append(c if w == 0 else d[w-1] + c)

        return ''.join(d)

* Test data compression and Calculate compression ratio

In [81]:
lz78 = LZ78_trie()

In [69]:
lz78.compress('infile.txt', 'compress.lz')

In [29]:
lz78.decompress('compress.lz', 'restore.txt')

In [30]:
import os

In [31]:
print('File name: infile.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%' % 
('{:,}'.format(os.path.getsize('./infile.txt')), '{:,}'.format(os.path.getsize('./compress.lz')), 
 os.path.getsize('./compress.lz')/os.path.getsize('./infile.txt')*100))

File name: infile.txt, 
Origin file size: 1,555,051Byte, 
Compressed size: 951,156Byte, 
Compression ratio: 61.165582%


In [120]:
lz78 = LZ78_trie()

In [121]:
lz78.compress('aladdin.txt', 'compress_aladdin')

In [122]:
lz78.decompress('compress_aladdin', 'aladdin_restore')